In [17]:
from frozendict import frozendict
from collections import abc

In [18]:
NUMERIC_METHODS = ("__add__", "__sub__", "__mul__", "__matmul__", "__truediv__", "__floordiv__", "__mod__", "__divmod__", "__pow__", "__lshift__", "__rshift__", "__and__", "__xor__", "__or__", "__rsub__", "__radd__")
STRING_METHODS = ('__add__', '__mul__', 'lower', '__rmul__', 'zfill', 'upper', 'translate', 'title', 'swapcase', 'strip', 'lstrip', 'rstrip', 'rjust', 'ljust', 'replace', )
PROPAGATE_LOOKUP = frozendict(
    (('string', STRING_METHODS),
     ('numeric', NUMERIC_METHODS),))

In [19]:
# I can't find this type so I make it here
NotImplementedType = type(int.__add__(1, 1.0))

In [20]:
class Disallowed(Exception):
    def __str__(self):
        return "This method has been disallowed"
class DisallowedMethod(Disallowed):
    def __init__(self, method_name, class_name):
        super().__init__(self)
        self.method_name = method_name
        
    def __str__(self):
        superstr = str(super().__str__(self))
        return "{superstr} for self.method_name"

In [40]:
def _add_class_method_propagation(cls, name):
    def newmeth(self, *args, **kwargs):
        """Closure to generates the supplied a magic method attribute for classes.
        for the current `name`.

        When called on the instance newmeth will return a value of the type of the instance
        instead of the parent class.

        Allow for any number of arguments."""
        supermeth = getattr(super(type(self), self), name)
        try:
            value = type(self)(supermeth(*args, **kwargs))
            return value
        except TypeError:
            dispargs = args
            if kwargs:
                dispargs.append(kwargs)
            error_msg = f"Could not generate {class_name_from_instance(self)}: for {name}: from given args: {args}"
            raise ValueError(error_msg) from None
    setattr(cls, name, newmeth)

In [41]:
def class_name_from_instance(inst):
    return type(inst).__name__

In [42]:
def _disallow_class_methods(cls, names):
    for name in names:
        def disallowed_attribute_function(*args, msg=name, **kwargs):
            """Decorator which returns a method which raise Disallowed
            with an optional custom message for any args."""
            def _not_implemented(*args, **kwargs):
                raise DisallowedMethod(name)
        setattr(cls, name, disallowed_attribute_function("Disallowed by type"))

In [43]:
def propagate(*names, magic_type=None, exclude=frozenset(), disallow=frozenset(), propagate_lookup=PROPAGATE_LOOKUP):
    """A decorator which adds subclass type propagation to method return values.
    
    This function is useful when subclassing types such as int when you want the magic methods
    to propagate the subclassed type.

    Args:
        *names:
            custom magic methods to propagate
    Keyword Only Arguments:
        magic_type:
            predefined sets of methods to do propagation on. Currently there is
            'numeric' and 'string'
            This is a useful alternative to specifying all the names you want.
        exclude:
            Exclude these method names from having a method-attribute set. Useful in
            conjunction with `magic_type`
        disallow:
            Disallow these methods names from being used at all.
    """
    # Input arguments flexibility
    names = set(names)
    if magic_type is not None:
        magic_type = magic_type.lower()
    if isinstance(exclude, str):
        exclude = set((exclude,))
    if isinstance(disallow, str):
        disallow = set((disallow,))
        
    # Add predefined collection of attribute if `magic_type` set
    # for usability
    if magic_type is not None:
        allowed_magic_types = set(propagate_lookup.keys())
        if magic_type not in allowed_magic_types:
            raise ValueError(f"Allowed values are: {','.join(allowed_magic_types)}")
        names.update(propagate_lookup[magic_type])
        
    # Exlclude attributes from being overridden
    names = names.difference(exclude)
    if isinstance(names, str):
        names = (names,)
    def apply_cls(cls):
        """sub decorator for setting class attribute-methods"""
        for name in names.difference(disallow):
            _add_class_method_propagation(cls, name)
        if disallow is not None:
            _disallow_class_methods(cls, disallow)
        return cls
    return apply_cls

In [44]:
1.0 == 1

True

In [45]:
class _mod:
    """Modular integer.
    
    Defaults to mod 3
    """
    default_mod = 3
    def __new__(cls, state, *, mod=None):
        #intstate = int(state)
        intstate = state
        if mod is None:
            mod = cls.default_mod
        try:
            intstate %= mod
        except TypeError as exc:
            raise TypeError(state, mod) from exc
        inst = super(_mod, cls).__new__(cls, intstate)
        return inst

In [46]:
@propagate(magic_type='numeric')
class modint(_mod, int):
    pass

@propagate(disallow='__truediv__')
class strictmodint(_mod, str):
    def __new__(cls, state):
        if not isinstance(state, int):
            raise ValueError(f"Must be an int: {state}")
        return super().__new__(cls, state)
    

In [47]:
@propagate(magic_type='string', disallow='__iter__')
class atomicstr(str):
    """Strings that disallow iteration."""
    pass

In [48]:
x = modint(5, mod=3)

In [49]:
x + 5

1

In [51]:
x - 1.0

ValueError: Could not generate modint: for __sub__: from given args: (1.0,)

In [52]:
for x in atomicstr("f"):
    print(x)

TypeError: 'atomicstr' object is not iterable